# Session 2: Pandas : Ingestion, Cleaning, and Aggregation

**Objective:** To gain insights in using pandas to load, clean, transform, and analyze data from various sources.

In [ ]:
! pip install pandas sqlalchemy pandas-datareader

## Part 1: Broad Concepts


### 1. Data Ingestion

What does pandas stands for?
Apparently:

*'The name is derived from the term "panel data", an econometrics term for data sets that include observations over multiple time periods for the same individuals,[3] as well as a play on the phrase "Python data analysis"'* (wikipedia).

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Ensure the necessary data files exist
!python3 database_setup.py # This runs the script to create the database and csv files

# From CSV
df_messy = pd.read_csv('messy_sales_data.csv')
print("--- Messy Sales Data Head ---")
print(df_messy.head())

# From SQL
engine = create_engine('sqlite:///company.db')
df_emp = pd.read_sql("SELECT * FROM employees", engine)
print("\n--- Employees Data Head ---")
print(df_emp.head())

Pandas allows to efficiently work on data, "à la Excel". 
The entire object is called a ```pandas DataFrame``` and the single columns are called ```Series```.


In [ ]:
print("\nData entries:\n", df_messy.keys())
print("\nData types:\n", df_messy.dtypes)

To select the column, use the column label (the one you see called by the ```keys()``` function above) in between square brackets [].

In [ ]:
print( df_messy['Price'], "\n\nand the type is\n\n", df_messy['Price'].dtype )

### 2. Data Cleaning & Preparation
This is the most time-consuming part of data analysis.

**Handling Missing Data (`NaN`)**

In [ ]:
# Check for missing values
print("Missing values before cleaning:")
print(df_messy.isnull().sum())


In [ ]:
# Fill numeric NaNs with the column median by reassigning the result
median_price = df_messy['Price'].median()
df_messy['Price'] = df_messy['Price'].fillna(median_price)

median_qty = df_messy['Quantity'].median()
df_messy['Quantity'] = df_messy['Quantity'].fillna(median_qty)



In [ ]:
# Fill categorical NaNs with a placeholder by reassigning the result
df_messy['Region'] = df_messy['Region'].fillna('Unknown')

In [ ]:
print("\nMissing values after cleaning:")
print(df_messy.isnull().sum())

print(df_messy)

**Strong-arm handling the Missing Data**:

The method ```dropna``` instead allows to delete completely the data where ```NaN``` are present. This can be done at very degrees of aggressiveness, for example:




In [ ]:
df_messy_again = pd.read_csv('messy_sales_data.csv')

In [ ]:
#Remove all entries with even a single NaN
print( df_messy_again.dropna() )

#Remove all columns with even a single NaN
print("\n", df_messy_again.dropna(axis='columns') )

#Remove all entries with NaNs but only in the column 'Price'
print("\n", df_messy_again.dropna(subset='Price') )


**Correcting Data Types**

In [ ]:
print("Data types before:", df_messy.dtypes)
df_messy['Quantity'] = df_messy['Quantity'].astype(int)
df_messy['Date'] = pd.to_datetime(df_messy['Date'])
print("\nData types after:", df_messy.dtypes)

**String Manipulation (`.str`)**

In [ ]:
print("Region values before cleanup:", df_messy['Region'].unique())
# Clean whitespace and standardize case
df_messy['Region'] = df_messy['Region'].str.strip().str.title()
print("\nRegion values after cleanup:", df_messy['Region'].unique())

**Handling Duplicates**

In [ ]:
print(f"Shape before dropping duplicates: {df_messy.shape}")
df_messy.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicates: {df_messy.shape}")

### 3. Data Transformation & Aggregation

**Applying Custom Functions (`.apply`)**

In [ ]:
def categorize_price(price):
    return 'High' if price > 200 else 'Low'

df_messy['PriceCategory'] = df_messy['Price'].apply(categorize_price)
print(df_messy[['Price', 'PriceCategory']].head())

**Grouping Data (`.groupby`)**: Essential for summary statistics.

In [ ]:
# Calculate average price per region
avg_price_by_region = df_messy.groupby('Region')['Price'].mean()
print(avg_price_by_region)

**Combining Datasets (`.merge`)**: SQL-style joins.

In [ ]:
df_dept = pd.read_csv('departments.csv')
# Merge employee data with department data
merged_df = pd.merge(df_emp, df_dept, on='department_id', how='left')
print(merged_df.head())

### 4. Basic Statistical Analysis 

**Statistical info of the dataframe (`.describe`)**:

In [ ]:
merged_df.describe()

In [ ]:
merged_df['salary'].describe()

**Performing other operations on data (`.agg`)**: 

In [ ]:
# Functions that return a single value
print( merged_df['salary'].agg(['mean', 'median']) )

# Function that return a transformation of the dataset
print( "\n", merged_df['salary'].agg(['cumsum', 'cumprod']) )

In [ ]:
# You cannot mix and match these two types...


print( "\n", merged_df['salary'].agg(['cumsum', 'sum']) )

## Part 2: Exercises 

### Lab 2.1: Comprehensive Data Cleanup and Analysis
* **Task:** Perform a full analysis pipeline on the company data.
  1. Load `employees.csv` and `departments.csv`.
  2. Merge the two DataFrames into one, using the department_id as the common key.
  3. Create a new column `salary_level` which is 'High' for salaries > 90000 and 'Standard' otherwise.
  4. Use `groupby()` to calculate the number of employees and the average salary for each `department_name`.
  5. Print the final report.

In [ ]:
import pandas as pd

# 1. Load data
df_emp = pd.read_csv('employees.csv')
df_dept = pd.read_csv('departments.csv')

# 2. Merge data

# 3. Create 'salary_level' column

# 4. Group and aggregate

# 5. Print report
print("Company Department Report:")
print(report)

### Lab 2.2: Playing around with real data
* **Task:** Perform exploratory analysis on real world economic data.
You will be given the code to use `pandas_reader` to download and clean a public available dataset of economic data.
  1. Explore how a real database is managed ( https://databank.worldbank.org/databases ) .
  2. Explore the numerics of the dataset, using the pandas internal correlation calculation to compute the correlation between gdp and life expectancy, and the correlation between the change of these indicators in time.

In [ ]:
from pandas_datareader import wb

# Define indicators and the time range
indicators = {'NY.GDP.PCAP.KD': 'gdp_per_capita', 'SP.DYN.LE00.IN': 'life_expectancy'}
start_year = 2019
end_year = 2023

# Fetch data from World Bank API
data = wb.download(indicator=list(indicators.keys()), 
                   country='all', 
                   start=start_year, 
                   end=end_year)

# Rename columns for clarity
data = data.rename(columns=indicators)

print("Original Data Sample:")
print(data.head())

In [ ]:
# Reset index to turn 'country' and 'year' into columns
df = data.reset_index()

# Drop rows with any missing values for our indicators
df_clean = df.dropna(subset=['gdp_per_capita', 'life_expectancy'])

# Display info and a sample of the cleaned data
print("\nCleaned Data Info:")
df_clean.info()

print("\nCleaned Data Sample:")
print(df_clean.head())

# Get country metadata to filter out aggregates
countries_info = wb.get_countries()
# Filter for countries only (regions have 'agg' in the 'region' column)
country_codes = countries_info[countries_info['region'] != 'Aggregates']['iso3c'].tolist()

# Filter our main dataframe to include only countries
df_countries = df_clean[df_clean['country'].isin(countries_info[countries_info['region'] != 'Aggregates']['name'])]


print(f"\nOriginal rows: {len(df_clean)}, Rows after filtering for countries: {len(df_countries)}")
